In [1]:
# Importing Libraries
from collections import defaultdict
from pathlib import Path
import numpy as np
import pandas as pd
from collections import namedtuple
from tabulate import tabulate

import torch
import os

from adversarial_debiasing import AdversarialDebiasing
from load_data import load_data, transform_data, Datapoint

from load_vectors import load_pretrained_vectors, load_vectors
import config
import utility_functions

import gensim
import gzip
import pickle

In [2]:
# For autoreloading changes made in other python scripts
%load_ext autoreload
%autoreload 2

In [3]:
# Loading the word vectors dictionary
word_vectors = load_pretrained_vectors(config.google_embedding_data_path, config.save_dir, config.google_embedding_type)

Loading from saved file.


In [4]:
# Load the google analogies training dataset:
analogy_dataset = load_data()
analogy_dataset

[Raw_Datapoint(x1='Athens', x2='Greece', x3='Baghdad', y='Iraq', task='capital-common-countries'),
 Raw_Datapoint(x1='Athens', x2='Greece', x3='Bangkok', y='Thailand', task='capital-common-countries'),
 Raw_Datapoint(x1='Athens', x2='Greece', x3='Beijing', y='China', task='capital-common-countries'),
 Raw_Datapoint(x1='Athens', x2='Greece', x3='Berlin', y='Germany', task='capital-common-countries'),
 Raw_Datapoint(x1='Athens', x2='Greece', x3='Bern', y='Switzerland', task='capital-common-countries'),
 Raw_Datapoint(x1='Athens', x2='Greece', x3='Cairo', y='Egypt', task='capital-common-countries'),
 Raw_Datapoint(x1='Athens', x2='Greece', x3='Canberra', y='Australia', task='capital-common-countries'),
 Raw_Datapoint(x1='Athens', x2='Greece', x3='Hanoi', y='Vietnam', task='capital-common-countries'),
 Raw_Datapoint(x1='Athens', x2='Greece', x3='Havana', y='Cuba', task='capital-common-countries'),
 Raw_Datapoint(x1='Athens', x2='Greece', x3='Helsinki', y='Finland', task='capital-common-cou

In [5]:
# Transform the data such that it includes the embeddings of the words in consideration
transformed_analogy_dataset, gender_subspace = transform_data(word_vectors, analogy_dataset, use_boluk = False)
word_embedding_dim = transformed_analogy_dataset[0].gt_embedding.shape[0]

In [6]:
# Testing the transformed analogy dataset
print(transformed_analogy_dataset[0].analogy_embeddings.shape)
print(transformed_analogy_dataset[0].gt_embedding.shape)
print(transformed_analogy_dataset[0].protected.shape)

(900,)
(300,)
(1,)


In [ ]:
# To run the grid-search and obtain the np.dot(w.T, g) values
learning_rate_list = [2 ** -12, 2 ** -6, 2 ** -3]
adversary_loss_weight_list = [1.0, 0.5, 0.1]
embedding_types = ["GoogleNews", "WikipediaVec", "Glove"]
for embedding_type in embeddings_types:
    word_vectors = load_pretrained_vectors(path, save_dir, embedding_type)
    transformed_analogy_dataset, gender_subspace = transform_data(word_vectors, analogy_dataset, use_boluk = False)
    word_embedding_dim = transformed_analogy_dataset[0].gt_embedding.shape[0]
    for learning_rate in learning_rate_list:
        for adversary_loss_weight in adversary_loss_weight_list:

            # Reset seeds
            np.random.seed(42)
            torch.manual_seed(42)
            torch.backends.cudnn.deterministic = True
            torch.backends.cudnn.benchmark = False
            
            non_debiased_model = AdversarialDebiasing(
                seed = 42,
                word_embedding_dim = word_embedding_dim,
                num_epochs = 500,
                debias = False,
                gender_subspace = gender_subspace,
                batch_size = 256,
                adversary_loss_weight = adversary_loss_weight,
                classifier_learning_rate = learning_rate,
                adversary_learning_rate = learning_rate
            )

            non_debiased_model.fit(dataset=transformed_analogy_dataset)

            temp_dict = {"W1" : non_debiased_model.best_W1, "W2" : non_debiased_model.best_W2}

            pickle.dump(temp_dict, os.path.join('models', "non_debiased", word_embedding_type + "_" + str(learning_rate) + "_" + str(adversary_loss_weight) + '.pckl'))

            pickle.dump(temp_dict, os.path.join('models', "non_debiased", word_embedding_type + "_" + str(learning_rate) + "_" + str(adversary_loss_weight) + '_last.pckl'))

            # Reset seeds
            np.random.seed(42)
            torch.manual_seed(42)
            torch.backends.cudnn.deterministic = True
            torch.backends.cudnn.benchmark = False
            
            debiased_model = AdversarialDebiasing(
                seed = 42,
                word_embedding_dim = word_embedding_dim,
                num_epochs = 500,
                debias = True,
                gender_subspace = gender_subspace,
                batch_size = 256,
                adversary_loss_weight = adversary_loss_weight,
                classifier_learning_rate = learning_rate,
                adversary_learning_rate = learning_rate
            )

            debiased_model.fit(dataset=transformed_analogy_dataset)

            temp_dict = {"W1" : debiased_model.best_W1, "W2" : debiased_model.best_W2}

            pickle.dump(temp_dict, os.path.join('models', "debiased", word_embedding_type + "_" + str(learning_rate) + "_" + str(adversary_loss_weight) + '.pckl'))

            pickle.dump(temp_dict, os.path.join('models', "debiased", word_embedding_type + "_" + str(learning_rate) + "_" + str(adversary_loss_weight) + '_last.pckl'))

In [ ]:
with open(os.path.join('models', 'non_debiased'))
non_debiased_model = AdversarialDebiasing(
                seed = 42,
                word_embedding_dim = word_embedding_dim,
                num_epochs = 500,
                debias = False,
                gender_subspace = gender_subspace,
                batch_size = 256,
                adversary_loss_weight = 0.1,
                classifier_learning_rate = 2 ** -6,
                adversary_learning_rate = 2 ** -6
            )
non_debiased_model.W1 = 

In [ ]:
word_embedding_dim = transformed_analogy_dataset[0].gt_embedding.shape[0]
# Training the variant of the model without debiasing
non_debiased_model = AdversarialDebiasing(
    word_embedding_dim=word_embedding_dim,
    num_epochs=500,
    debias=False,
    gender_subspace=gender_subspace,
    batch_size=256,
    adversary_loss_weight=0.1,
    classifier_learning_rate = 2 ** -6,
    adversary_learning_rate = 2 ** -6
)
non_debiased_model.fit(dataset=transformed_analogy_dataset)

In [ ]:
W1 = non_debiased_model.get_model_weights()
print(np.dot(W1.detach().numpy().T,gender_subspace.T))

In [ ]:
gender_subspace.shape

In [ ]:
# Training the variant of the model with debiasing
# debiased_model = AdversarialDebiasing(
#     word_embedding_dim=word_embedding_dim,
#     num_epochs=500,
#     debias=True,
#     gender_subspace=gender_subspace,
#     batch_size=256,
#     adversary_loss_weight=0.1,
#     classifier_learning_rate = 2 ** -8,
#     adversary_learning_rate = 2 ** -8
# )
debiased_model = AdversarialDebiasing(
    word_embedding_dim=word_embedding_dim,
    num_epochs=500,
    debias=True,
    gender_subspace=gender_subspace,
    batch_size=256,
    adversary_loss_weight=0.1,
    classifier_learning_rate = 2 ** -6,
    adversary_learning_rate = 2 ** -6
)

debiased_model.fit(dataset=transformed_analogy_dataset)

In [ ]:
W1 = debiased_model.get_model_weights()
print(np.dot(W1.clone().cpu().detach().numpy().T,gender_subspace.T))

In [ ]:
# Examples to test the models upon
datapoints, test_analogies = [], []
with open(os.path.join('data', 'sexism-traps.txt'), 'r') as f:
    # Reading each line
    for line in f.readlines():
        words = line.split()
        if words[0] == ':':
            continue
        test_analogies.append(words)
        word_embeddings = word_vectors[words]
        word_embeddings = np.reshape(word_embeddings, (1, -1))
        datapoints.append(word_embeddings)
datapoints = np.vstack(datapoints)
print(datapoints.shape)

In [ ]:
# Qualitative evaluation of the non-debiased model

# non_debiased_predictions = non_debiased_model.predict(datapoints)
features = torch.cat([torch.Tensor(x).unsqueeze_(0) for x in datapoints])
x1 = features[:, 0:word_embedding_dim]
x2 = features[:, word_embedding_dim:word_embedding_dim * 2]
x3 = features[:, word_embedding_dim * 2:word_embedding_dim * 3]

non_debiased_predictions = x2 + x3 - x1
non_debiased_predictions = non_debiased_predictions.cpu().detach().numpy()

In [ ]:
non_debiased_most_similar_list = utility_functions.obtain_most_similar(non_debiased_predictions, word_vectors)

In [ ]:
# Displaying the similarity list for the non-debiased model
non_debiased_most_similar_list_data_frames = []
for i in range(len(non_debiased_most_similar_list)):
    # print("{} : {} :: {} : ".format(test_analogies[i][0], test_analogies[i][1], test_analogies[i][2]))
    temp_data_frame = pd.DataFrame(non_debiased_most_similar_list[i][1:], columns = ['Neighbor', 'Similarity'])
    non_debiased_most_similar_list_data_frames.append(temp_data_frame)
    # print(tabulate(temp_data_frame, headers='keys', tablefmt='psql', showindex=False))

In [ ]:
# Qualitative evaluation of the debiased model
debiased_predictions = debiased_model.predict(datapoints)
debiased_most_similar_list = utility_functions.obtain_most_similar(debiased_predictions, word_vectors)

In [ ]:
# Displaying the similarity list for the debiased model
debiased_most_similar_list_data_frames = []
for i in range(len(debiased_most_similar_list)):
    # print("{} : {} :: {} : ".format(test_analogies[i][0], test_analogies[i][1], test_analogies[i][2]))
    temp_data_frame = pd.DataFrame(debiased_most_similar_list[i][1:], columns = ['Neighbor', 'Similarity'])
    debiased_most_similar_list_data_frames.append(temp_data_frame)
    # print(tabulate(temp_data_frame, headers='keys', tablefmt='psql', showindex=False))

In [ ]:
# Combining the dataframes pertaining to both the variants of the model
iterables = [['Biased', 'Debiased'], ['Neighbour', 'Similarity']]
index = pd.MultiIndex.from_product(iterables)
overall_data_frames_list = []
for i in range(len(non_debiased_most_similar_list)):
    overall_list = []
    print("{} : {} :: {} : ".format(test_analogies[i][0], test_analogies[i][1], test_analogies[i][2]))
    for j in range(1, len(non_debiased_most_similar_list[i])):
        temp_list = []
        temp_list.append(non_debiased_most_similar_list[i][j][0])
        temp_list.append(round(non_debiased_most_similar_list[i][j][1], 3))
        temp_list.append(debiased_most_similar_list[i][j][0])
        temp_list.append(round(debiased_most_similar_list[i][j][1], 3))
        overall_list.append(temp_list)
    temp_df = pd.DataFrame(overall_list, columns = index)
    # print(temp_df.to_string(index = False))
    print(tabulate(temp_df, headers = ['Biased\nNeighbour', 'Biased\nSimilarity', 'Debiased\nNeighbour', 'Debiased\nSimilarity'], tablefmt = 'psql', showindex = False))
    overall_data_frames_list.append(temp_df)